In [2]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [3]:
# SOME HELPER FUNCTIONS
def date_time(table_cells):
    """
    This function returns the data and time from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [3]:
# static link for convenience (updated June 9, 2021)
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [4]:
# Perform a get request on the provided static_url
## and assign the response to a object

response = requests.get(static_url)

# Check the status code
response.status_code

200

In [ ]:
# We create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, 'html.parser')

# Print the title of the page to check if it was correctly loaded
soup.title

#dir(soup) # command to check the existing attributes

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [6]:
# print the soup object
print(soup.prettify()[:1000])

# Use the find_all function in the BeautifulSoup object,
 # with element type `table`. Assign the result to a list called `html_tables`
html_tables = soup.find_all(name = 'table')
print(f"\nFound {len(html_tables)} tables on the page")


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Falcon 9 and Falcon Heavy launches - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-fe

In [10]:
# A quick look show that the interesting data starts at the 3rd table
first_launch_table= html_tables[2]
#print(first_launch_table.prettify())

# initialize list of columns
column_names = []

# Apply find_all() function with `th` element on first_launch_table to extract column names one by one.
first_launch_table.find_all(name='th')

# Iterate each th element and apply the provided extract_column_from_header() to get a column name
for th_element in first_launch_table.find_all('th'):
    name = extract_column_from_header(th_element)
    if name is not None and len(name) >0:
        # Append the Non-empty column name into our list
        column_names.append(name)


#check the extracted column names
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-0">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-0">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
      c
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Payload mass
   </th>
   <th scope="col">
    Orbit
   </th>
   <th scope="col">
    Customer
   </th>
   <th scope="col">
    Launch
    <br/>
    outcome
   </th>
   <th scope="col">
    <a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">
     Booster
     <br/>
     landing
    </a>
   </th>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    1
   </th>
   <td>
    4 June 2010,
    <br/>
    18:45
   </td>
   <td>
    <a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">
     F9 v1.0
    </a>
    <sup class="reference" id="cite_ref-MuskMay2012_13-0">
     <a href="#cite_note-MuskMay2012-13">
      <span class="cite-bracket">
       [
      </span>
      7
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    B0003.1
    <sup class="reference" id="cite_ref-block_numbers_14-0">
     <a href="#cite_note-block_numbers-14">
      <span class="cite-bracket">
       [
      </span>
      8
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCAFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/Dragon_Spacecraft_Qualification_Unit" title="Dragon Spacecraft Qualification Unit">
     Dragon Spacecraft Qualification Unit
    </a>
   </td>
   <td>
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-failure" style="background: #FFC7C7; color:black; vertical-align: middle; text-align: center;">
    Failure
    <sup class="reference" id="cite_ref-ns20110930_15-0">
     <a href="#cite_note-ns20110930-15">
      <span class="cite-bracket">
       [
      </span>
      9
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-16">
     <a href="#cite_note-16">
      <span class="cite-bracket">
       [
      </span>
      10
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    <small>
     (parachute)
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    First flight of Falcon 9 v1.0.
    <sup class="reference" id="cite_ref-sfn20100604_17-0">
     <a href="#cite_note-sfn20100604-17">
      <span class="cite-bracket">
       [
      </span>
      11
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    Used a boilerplate version of Dragon capsule which was not designed to separate from the second stage.
    <small>
     (
     <a href="#First_flight_of_Falcon_9">
      more details below
     </a>
     )
    </small>
    Attempted to recover the first stage by parachuting it into the ocean, but it burned up on reentry, before the parachutes even deployed.
    <sup class="reference" id="cite_ref-parachute_18-0">
     <a href="#cite_note-parachute-18">
      <span class="cite-bracket">
       [
      </span>
      12
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    2
   </th>
   <td>
    8 December 2010,
    <br/>
    15:43
    <sup class="reference" id="cite_ref-spaceflightnow_Clark_Launch_Report_19-0">
     <a href="#cite_note-spaceflightnow_Clark_Launch_Report-19">
      <span class="cite-bracket">
       [
      </span>
      13
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">
     F9 v1.0
    </a>
    <sup class="reference" id="cite_ref-MuskMay2012_13-1">
     <a href="#cite_note-MuskMay2012-13">
      <span class="cite-bracket">
       [
      </span>
      7
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    B0004.1
    <sup class="reference" id="cite_ref-block_numbers_14-1">
     <a href="#cite_note-block_numbers-14">
      <span class="cite-bracket">
       [
      </span>
      8
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCAFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX_Dragon" title="SpaceX Dragon">
     Dragon
    </a>
    <a class="mw-redirect" href="/wiki/COTS_Demo_Flight_1" title="COTS Demo Flight 1">
     demo flight C1
    </a>
    <br/>
    (Dragon C101)
   </td>
   <td>
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
    (
    <a href="/wiki/International_Space_Station" title="International Space Station">
     ISS
    </a>
    )
   </td>
   <td>
    <style data-mw-deduplicate="TemplateStyles:r1126788409">
     .mw-parser-output .plainlist ol,.mw-parser-output .plainlist ul{line-height:inherit;list-style:none;margin:0;padding:0}.mw-parser-output .plainlist ol li,.mw-parser-output .plainlist ul li{margin-bottom:0}
    </style>
    <div class="plainlist">
     <ul>
      <li>
       <a href="/wiki/NASA" title="NASA">
        NASA
       </a>
       (
       <a href="/wiki/Commercial_Orbital_Transportation_Services" title="Commercial Orbital Transportation Services">
        COTS
       </a>
       )
      </li>
      <li>
       <a href="/wiki/National_Reconnaissance_Office" title="National Reconnaissance Office">
        NRO
       </a>
      </li>
     </ul>
    </div>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <sup class="reference" id="cite_ref-ns20110930_15-1">
     <a href="#cite_note-ns20110930-15">
      <span class="cite-bracket">
       [
      </span>
      9
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td class="table-failure" style="background: #FFC7C7; color:black; vertical-align: middle; text-align: center;">
    Failure
    <sup class="reference" id="cite_ref-ns20110930_15-2">
     <a href="#cite_note-ns20110930-15">
      <span class="cite-bracket">
       [
      </span>
      9
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-20">
     <a href="#cite_note-20">
      <span class="cite-bracket">
       [
      </span>
      14
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    <small>
     (parachute)
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Maiden flight of
    <a class="mw-redirect" href="/wiki/Dragon_capsule" title="Dragon capsule">
     Dragon capsule
    </a>
    , consisting of over 3 hours of testing thruster maneuvering and reentry.
    <sup class="reference" id="cite_ref-spaceflightnow_Clark_unleashing_Dragon_21-0">
     <a href="#cite_note-spaceflightnow_Clark_unleashing_Dragon-21">
      <span class="cite-bracket">
       [
      </span>
      15
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    Attempted to recover the first stage by parachuting it into the ocean, but it disintegrated upon reentry, before the parachutes were deployed.
    <sup class="reference" id="cite_ref-parachute_18-1">
     <a href="#cite_note-parachute-18">
      <span class="cite-bracket">
       [
      </span>
      12
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <small>
     (
     <a href="#COTS_demo_missions">
      more details below
     </a>
     )
    </small>
    It also included two
    <a href="/wiki/CubeSat" title="CubeSat">
     CubeSats
    </a>
    ,
    <sup class="reference" id="cite_ref-NRO_Taps_Boeing_for_Next_Batch_of_CubeSats_22-0">
     <a href="#cite_note-NRO_Taps_Boeing_for_Next_Batch_of_CubeSats-22">
      <span class="cite-bracket">
       [
      </span>
      16
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    and a wheel of
    <a href="/wiki/Brou%C3%A8re" title="Brouère">
     Brouère
    </a>
    cheese.
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    3
   </th>
   <td>
    22 May 2012,
    <br/>
    07:44
    <sup class="reference" id="cite_ref-BBC_new_era_23-0">
     <a href="#cite_note-BBC_new_era-23">
      <span class="cite-bracket">
       [
      </span>
      17
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">
     F9 v1.0
    </a>
    <sup class="reference" id="cite_ref-MuskMay2012_13-2">
     <a href="#cite_note-MuskMay2012-13">
      <span class="cite-bracket">
       [
      </span>
      7
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    B0005.1
    <sup class="reference" id="cite_ref-block_numbers_14-2">
     <a href="#cite_note-block_numbers-14">
      <span class="cite-bracket">
       [
      </span>
      8
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCAFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX_Dragon" title="SpaceX Dragon">
     Dragon
    </a>
    <a class="mw-redirect" href="/wiki/Dragon_C2%2B" title="Dragon C2+">
     demo flight C2+
    </a>
    <sup class="reference" id="cite_ref-C2_24-0">
     <a href="#cite_note-C2-24">
      <span class="cite-bracket">
       [
      </span>
      18
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    (Dragon C102)
   </td>
   <td>
    525 kg (1,157 lb)
    <sup class="reference" id="cite_ref-25">
     <a href="#cite_note-25">
      <span class="cite-bracket">
       [
      </span>
      19
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
    (
    <a href="/wiki/International_Space_Station" title="International Space Station">
     ISS
    </a>
    )
   </td>
   <td>
    <a href="/wiki/NASA" title="NASA">
     NASA
    </a>
    (
    <a href="/wiki/Commercial_Orbital_Transportation_Services" title="Commercial Orbital Transportation Services">
     COTS
    </a>
    )
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <sup class="reference" id="cite_ref-26">
     <a href="#cite_note-26">
      <span class="cite-bracket">
       [
      </span>
      20
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td class="table-noAttempt" style="background: #EEE; color:black; vertical-align: middle; white-space: nowrap; text-align: center;">
    No attempt
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Dragon spacecraft demonstrated a series of tests before it was allowed to approach the
    <a href="/wiki/International_Space_Station" title="International Space Station">
     International Space Station
    </a>
    . Two days later, it became the first commercial spacecraft to board the ISS.
    <sup class="reference" id="cite_ref-BBC_new_era_23-1">
     <a href="#cite_note-BBC_new_era-23">
      <span class="cite-bracket">
       [
      </span>
      17
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <small>
     (
     <a href="#COTS_demo_missions">
      more details below
     </a>
     )
    </small>
   </td>
  </tr>
  <tr>
   <th rowspan="3" scope="row" style="text-align:center;">
    4
   </th>
   <td rowspan="2">
    8 October 2012,
    <br/>
    00:35
    <sup class="reference" id="cite_ref-SFN_LLog_27-0">
     <a href="#cite_note-SFN_LLog-27">
      <span class="cite-bracket">
       [
      </span>
      21
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td rowspan="2">
    <a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">
     F9 v1.0
    </a>
    <sup class="reference" id="cite_ref-MuskMay2012_13-3">
     <a href="#cite_note-MuskMay2012-13">
      <span class="cite-bracket">
       [
      </span>
      7
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    B0006.1
    <sup class="reference" id="cite_ref-block_numbers_14-3">
     <a href="#cite_note-block_numbers-14">
      <span class="cite-bracket">
       [
      </span>
      8
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td rowspan="2">
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCAFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX_CRS-1" title="SpaceX CRS-1">
     SpaceX CRS-1
    </a>
    <sup class="reference" id="cite_ref-sxManifest20120925_28-0">
     <a href="#cite_note-sxManifest20120925-28">
      <span class="cite-bracket">
       [
      </span>
      22
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    (Dragon C103)
   </td>
   <td>
    4,700 kg (10,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
    (
    <a href="/wiki/International_Space_Station" title="International Space Station">
     ISS
    </a>
    )
   </td>
   <td>
    <a href="/wiki/NASA" title="NASA">
     NASA
    </a>
    (
    <a href="/wiki/Commercial_Resupply_Services" title="Commercial Resupply Services">
     CRS
    </a>
    )
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td rowspan="2" style="background:#ececec; text-align:center;">
    <span class="nowrap">
     No attempt
    </span>
   </td>
  </tr>
  <tr>
   <td>
    <a href="/wiki/Orbcomm_(satellite)" title="Orbcomm (satellite)">
     Orbcomm-OG2
    </a>
    <sup class="reference" id="cite_ref-Orbcomm_29-0">
     <a href="#cite_note-Orbcomm-29">
      <span class="cite-bracket">
       [
      </span>
      23
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    172 kg (379 lb)
    <sup class="reference" id="cite_ref-gunter-og2_30-0">
     <a href="#cite_note-gunter-og2-30">
      <span class="cite-bracket">
       [
      </span>
      24
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/Orbcomm" title="Orbcomm">
     Orbcomm
    </a>
   </td>
   <td class="table-partial" style="background: #FFB; color:black; vertical-align: middle; text-align: center;">
    Partial failure
    <sup class="reference" id="cite_ref-nyt-20121030_31-0">
     <a href="#cite_note-nyt-20121030-31">
      <span class="cite-bracket">
       [
      </span>
      25
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    CRS-1 was successful, but the
    <a href="/wiki/Secondary_payload" title="Secondary payload">
     secondary payload
    </a>
    was inserted into an abnormally low orbit and subsequently lost. This was due to one of the nine
    <a href="/wiki/SpaceX_Merlin" title="SpaceX Merlin">
     Merlin engines
    </a>
    shutting down during the launch, and NASA declining a second reignition, as per
    <a href="/wiki/International_Space_Station" title="International Space Station">
     ISS
    </a>
    visiting vehicle safety rules, the primary payload owner is contractually allowed to decline a second reignition. NASA stated that this was because SpaceX could not guarantee a high enough likelihood of the second stage completing the second burn successfully which was required to avoid any risk of secondary payload's collision with the ISS.
    <sup class="reference" id="cite_ref-OrbcommTotalLoss_32-0">
     <a href="#cite_note-OrbcommTotalLoss-32">
      <span class="cite-bracket">
       [
      </span>
      26
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-sn20121011_33-0">
     <a href="#cite_note-sn20121011-33">
      <span class="cite-bracket">
       [
      </span>
      27
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-34">
     <a href="#cite_note-34">
      <span class="cite-bracket">
       [
      </span>
      28
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    5
   </th>
   <td>
    1 March 2013,
    <br/>
    15:10
   </td>
   <td>
    <a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">
     F9 v1.0
    </a>
    <sup class="reference" id="cite_ref-MuskMay2012_13-4">
     <a href="#cite_note-MuskMay2012-13">
      <span class="cite-bracket">
       [
      </span>
      7
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    B0007.1
    <sup class="reference" id="cite_ref-block_numbers_14-4">
     <a href="#cite_note-block_numbers-14">
      <span class="cite-bracket">
       [
      </span>
      8
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCAFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX_CRS-2" title="SpaceX CRS-2">
     SpaceX CRS-2
    </a>
    <sup class="reference" id="cite_ref-sxManifest20120925_28-1">
     <a href="#cite_note-sxManifest20120925-28">
      <span class="cite-bracket">
       [
      </span>
      22
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    (Dragon C104)
   </td>
   <td>
    4,877 kg (10,752 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
    (
    <a class="mw-redirect" href="/wiki/ISS" title="ISS">
     ISS
    </a>
    )
   </td>
   <td>
    <a href="/wiki/NASA" title="NASA">
     NASA
    </a>
    (
    <a href="/wiki/Commercial_Resupply_Services" title="Commercial Resupply Services">
     CRS
    </a>
    )
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-noAttempt" style="background: #EEE; color:black; vertical-align: middle; white-space: nowrap; text-align: center;">
    No attempt
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Last launch of the original Falcon 9 v1.0
    <a href="/wiki/Launch_vehicle" title="Launch vehicle">
     launch vehicle
    </a>
    , first use of the unpressurized trunk section of Dragon.
    <sup class="reference" id="cite_ref-sxf9_20110321_35-0">
     <a href="#cite_note-sxf9_20110321-35">
      <span class="cite-bracket">
       [
      </span>
      29
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    6
   </th>
   <td>
    29 September 2013,
    <br/>
    16:00
    <sup class="reference" id="cite_ref-pa20130930_36-0">
     <a href="#cite_note-pa20130930-36">
      <span class="cite-bracket">
       [
      </span>
      30
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_v1.1" title="Falcon 9 v1.1">
     F9 v1.1
    </a>
    <sup class="reference" id="cite_ref-MuskMay2012_13-5">
     <a href="#cite_note-MuskMay2012-13">
      <span class="cite-bracket">
       [
      </span>
      7
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <br/>
    B1003
    <sup class="reference" id="cite_ref-block_numbers_14-5">
     <a href="#cite_note-block_numbers-14">
      <span class="cite-bracket">
       [
      </span>
      8
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a class="mw-redirect" href="/wiki/Vandenberg_Air_Force_Base" title="Vandenberg Air Force Base">
     VAFB
    </a>
    ,
    <br/>
    <a href="/wiki/Vandenberg_Space_Launch_Complex_4" title="Vandenberg Space Launch Complex 4">
     SLC-4E
    </a>
   </td>
   <td>
    <a href="/wiki/CASSIOPE" title="CASSIOPE">
     CASSIOPE
    </a>
    <sup class="reference" id="cite_ref-sxManifest20120925_28-2">
     <a href="#cite_note-sxManifest20120925-28">
      <span class="cite-bracket">
       [
      </span>
      22
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-CASSIOPE_MDA_37-0">
     <a href="#cite_note-CASSIOPE_MDA-37">
      <span class="cite-bracket">
       [
      </span>
      31
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    500 kg (1,100 lb)
   </td>
   <td>
    <a href="/wiki/Polar_orbit" title="Polar orbit">
     Polar orbit
    </a>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/Maxar_Technologies" title="Maxar Technologies">
     MDA
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <sup class="reference" id="cite_ref-pa20130930_36-1">
     <a href="#cite_note-pa20130930-36">
      <span class="cite-bracket">
       [
      </span>
      30
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td class="table-no2" style="background: #FFE3E3; color: black; vertical-align: middle; text-align: center;">
    Uncontrolled
    <br/>
    <small>
     (ocean)
    </small>
    <sup class="reference" id="cite_ref-ocean_landing_38-0">
     <a href="#cite_note-ocean_landing-38">
      <span class="cite-bracket">
       [
      </span>
      d
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    First commercial mission with a private customer, first launch from Vandenberg, and demonstration flight of Falcon 9 v1.1 with an improved 13-tonne to LEO capacity.
    <sup class="reference" id="cite_ref-sxf9_20110321_35-1">
     <a href="#cite_note-sxf9_20110321-35">
      <span class="cite-bracket">
       [
      </span>
      29
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    After separation from the second stage carrying Canadian commercial and scientific satellites, the first stage booster performed a controlled reentry,
    <sup class="reference" id="cite_ref-39">
     <a href="#cite_note-39">
      <span class="cite-bracket">
       [
      </span>
      32
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    and an
    <a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">
     ocean touchdown test
    </a>
    for the first time. This provided good test data, even though the booster started rolling as it neared the ocean, leading to the shutdown of the central engine as the roll depleted it of fuel, resulting in a hard impact with the ocean.
    <sup class="reference" id="cite_ref-pa20130930_36-2">
     <a href="#cite_note-pa20130930-36">
      <span class="cite-bracket">
       [
      </span>
      30
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    This was the first known attempt of a rocket engine being lit to perform a supersonic retro propulsion, and allowed SpaceX to enter a public-private partnership with
    <a href="/wiki/NASA" title="NASA">
     NASA
    </a>
    and its Mars entry, descent, and landing technologies research projects.
    <sup class="reference" id="cite_ref-40">
     <a href="#cite_note-40">
      <span class="cite-bracket">
       [
      </span>
      33
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <small>
     (
     <a href="#Maiden_flight_of_v1.1">
      more details below
     </a>
     )
    </small>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    7
   </th>
   <td>
    3 December 2013,
    <br/>
    22:41
    <sup class="reference" id="cite_ref-sfn_wwls20130624_41-0">
     <a href="#cite_note-sfn_wwls20130624-41">
      <span class="cite-bracket">
       [
      </span>
      34
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_v1.1" title="Falcon 9 v1.1">
     F9 v1.1
    </a>
    <br/>
    B1004
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCAFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/SES-8" title="SES-8">
     SES-8
    </a>
    <sup class="reference" id="cite_ref-sxManifest20120925_28-3">
     <a href="#cite_note-sxManifest20120925-28">
      <span class="cite-bracket">
       [
      </span>
      22
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-spx-pr_42-0">
     <a href="#cite_note-spx-pr-42">
      <span class="cite-bracket">
       [
      </span>
      35
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-aw20110323_43-0">
     <a href="#cite_note-aw20110323-43">
      <span class="cite-bracket">
       [
      </span>
      36
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    3,170 kg (6,990 lb)
   </td>
   <td>
    <a href="/wiki/Geostationary_transfer_orbit" title="Geostationary transfer orbit">
     GTO
    </a>
   </td>
   <td>
    <a class="mw-redirect" href="/wiki/SES_S.A." title="SES S.A.">
     SES
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <sup class="reference" id="cite_ref-SNMissionStatus7_44-0">
     <a href="#cite_note-SNMissionStatus7-44">
      <span class="cite-bracket">
       [
      </span>
      37
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td class="table-noAttempt" style="background: #EEE; color:black; vertical-align: middle; white-space: nowrap; text-align: center;">
    No attempt
    <br/>
    <sup class="reference" id="cite_ref-sf10120131203_45-0">
     <a href="#cite_note-sf10120131203-45">
      <span class="cite-bracket">
       [
      </span>
      38
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    First
    <a href="/wiki/Geostationary_transfer_orbit" title="Geostationary transfer orbit">
     Geostationary transfer orbit
    </a>
    (GTO) launch for Falcon 9,
    <sup class="reference" id="cite_ref-spx-pr_42-1">
     <a href="#cite_note-spx-pr-42">
      <span class="cite-bracket">
       [
      </span>
      35
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    and first successful reignition of the second stage.
    <sup class="reference" id="cite_ref-46">
     <a href="#cite_note-46">
      <span class="cite-bracket">
       [
      </span>
      39
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    SES-8 was inserted into a
    <a href="/wiki/Geostationary_transfer_orbit" title="Geostationary transfer orbit">
     Super-Synchronous Transfer Orbit
    </a>
    of 79,341 km (49,300 mi) in apogee with an
    <a href="/wiki/Orbital_inclination" title="Orbital inclination">
     inclination
    </a>
    of 20.55° to the
    <a href="/wiki/Equator" title="Equator">
     equator
    </a>
    .
   </td>
  </tr>
 </tbody>
</table>



In [ ]:
# The eleventh table happens to be the last launch table

eleventh_launch_table= html_tables[10]
print(eleventh_launch_table.prettify())

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight
    <p>
     No.
    </p>
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-8">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch
    <br/>
    site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-8">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
      c
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Payload mass
   </th>
   <th scope="col">
    Orbit
   </th>
   <th scope="col">
    Customer
   </th>
   <th scope="col">
    Launch
    <br/>
    outcome
   </th>
   <th scope="col">
    <a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">
     Booster
     <br/>
     landing
    </a>
   </th>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    104
   </th>
   <td>
    8 January 2021
    <br/>
    02:15
    <sup class="reference" id="cite_ref-616">
     <a href="#cite_note-616">
      <span class="cite-bracket">
       [
      </span>
      604
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1060" title="List of Falcon 9 first-stage boosters">
     B1060.4
    </a>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/T%C3%BCrksat_5A" title="Türksat 5A">
     Türksat 5A
    </a>
    <sup class="reference" id="cite_ref-turk5a_617-0">
     <a href="#cite_note-turk5a-617">
      <span class="cite-bracket">
       [
      </span>
      605
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    3,500 kg (7,700 lb)
   </td>
   <td>
    <a href="/wiki/Geostationary_transfer_orbit" title="Geostationary transfer orbit">
     GTO
    </a>
   </td>
   <td>
    <a href="/wiki/T%C3%BCrksat_(company)" title="Türksat (company)">
     Türksat
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    A 3,500 kg (7,700 lb) satellite intended to be stationed at 31.0° east.
    <sup class="reference" id="cite_ref-turk5a_617-1">
     <a href="#cite_note-turk5a-617">
      <span class="cite-bracket">
       [
      </span>
      605
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    This is the most powerful satellite in Türksat's fleet
    <sup class="reference" id="cite_ref-618">
     <a href="#cite_note-618">
      <span class="cite-bracket">
       [
      </span>
      606
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    and will provide
    <a class="mw-redirect" href="/wiki/Ku-band" title="Ku-band">
     Ku-band
    </a>
    television broadcast services over
    <a href="/wiki/Turkey" title="Turkey">
     Turkey
    </a>
    , the
    <a href="/wiki/Middle_East" title="Middle East">
     Middle East
    </a>
    ,
    <a href="/wiki/Europe" title="Europe">
     Europe
    </a>
    and
    <a href="/wiki/Africa" title="Africa">
     Africa
    </a>
    . The satellite was injected in to a
    <a href="/wiki/Geostationary_transfer_orbit" title="Geostationary transfer orbit">
     Super-synchronous transfer orbit
    </a>
    of 280 km × 55,000 km (170 mi × 34,180 mi) with 17.6°
    <a href="/wiki/Orbital_inclination" title="Orbital inclination">
     inclination
    </a>
    .
    <sup class="reference" id="cite_ref-619">
     <a href="#cite_note-619">
      <span class="cite-bracket">
       [
      </span>
      607
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    105
   </th>
   <td>
    20 January 2021
    <br/>
    13:02
    <sup class="reference" id="cite_ref-620">
     <a href="#cite_note-620">
      <span class="cite-bracket">
       [
      </span>
      608
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1051" title="List of Falcon 9 first-stage boosters">
     B1051.8
    </a>
    <sup class="reference" id="cite_ref-621">
     <a href="#cite_note-621">
      <span class="cite-bracket">
       [
      </span>
      609
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Kennedy_Space_Center" title="Kennedy Space Center">
     KSC
    </a>
    ,
    <br/>
    <a href="/wiki/Kennedy_Space_Center_Launch_Complex_39A" title="Kennedy Space Center Launch Complex 39A">
     LC-39A
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    16 v1.0 (60 satellites)
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    The first booster to successfully launch and land eight times. Achieved a record turnaround time between two launches of the same booster of only 38 days and brought the total of launched Starlink satellites to over 1000.
    <sup class="reference" id="cite_ref-622">
     <a href="#cite_note-622">
      <span class="cite-bracket">
       [
      </span>
      610
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    SpaceX stated that the landing would occur during higher winds than usual; this test to expand the landing envelope was successfully passed by the booster.
    <sup class="reference" id="cite_ref-623">
     <a href="#cite_note-623">
      <span class="cite-bracket">
       [
      </span>
      611
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    106
   </th>
   <td>
    24 January 2021
    <br/>
    15:00
    <sup class="reference" id="cite_ref-624">
     <a href="#cite_note-624">
      <span class="cite-bracket">
       [
      </span>
      612
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1058" title="List of Falcon 9 first-stage boosters">
     B1058.5
    </a>
    <sup class="reference" id="cite_ref-625">
     <a href="#cite_note-625">
      <span class="cite-bracket">
       [
      </span>
      613
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <i>
     <a class="new" href="/w/index.php?title=Transporter-1&amp;action=edit&amp;redlink=1" title="Transporter-1 (page does not exist)">
      Transporter-1
     </a>
    </i>
    (
    <a href="/wiki/List_of_spaceflight_launches_in_January%E2%80%93June_2021#SpXTransporter1" title="List of spaceflight launches in January–June 2021">
     143 smallsat rideshare
    </a>
    )
   </td>
   <td>
    ~5,000 kg (11,000 lb)
   </td>
   <td>
    <a href="/wiki/Sun-synchronous_orbit" title="Sun-synchronous orbit">
     SSO
    </a>
   </td>
   <td>
    Various
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    First dedicated smallsat rideshare launch, targeting a 525 km (326 mi)
    <a class="mw-redirect" href="/wiki/Orbital_altitude" title="Orbital altitude">
     altitude
    </a>
    orbit.
    <sup class="reference" id="cite_ref-RsA_626-0">
     <a href="#cite_note-RsA-626">
      <span class="cite-bracket">
       [
      </span>
      614
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    The launch deployed a record 143 satellites, consisting of 120
    <a href="/wiki/CubeSat" title="CubeSat">
     CubeSats
    </a>
    , 11
    <a class="mw-redirect" href="/wiki/Microsatellite_(spaceflight)" title="Microsatellite (spaceflight)">
     microsatellites
    </a>
    , 10
    <a href="/wiki/Starlink" title="Starlink">
     Starlinks
    </a>
    , and 2 transfer stages. In addition, 2 hosted payloads and 1 non-separating dummy satellite
    <sup class="reference" id="cite_ref-627">
     <a href="#cite_note-627">
      <span class="cite-bracket">
       [
      </span>
      615
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    were
    <sup class="noprint Inline-Template" style="white-space:nowrap;">
     [
     <i>
      <a href="/wiki/Wikipedia:Verifiability" title="Wikipedia:Verifiability">
       <span title="The material near this tag failed verification of its source citation(s). (May 2021)">
        failed verification
       </span>
      </a>
     </i>
     ]
    </sup>
    launched.
    <sup class="reference" id="cite_ref-628">
     <a href="#cite_note-628">
      <span class="cite-bracket">
       [
      </span>
      616
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    These include
    <a class="mw-redirect" href="/wiki/SpaceBEE" title="SpaceBEE">
     SpaceBEE
    </a>
    (x 36),
    <a href="/wiki/Spire_Global" title="Spire Global">
     Lemur-2
    </a>
    (x 8),
    <a href="/wiki/ICEYE" title="ICEYE">
     ICEYE
    </a>
    (x 3), UVSQ-SAT,
    <sup class="reference" id="cite_ref-629">
     <a href="#cite_note-629">
      <span class="cite-bracket">
       [
      </span>
      617
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <a href="/wiki/Educational_Launch_of_Nanosatellites" title="Educational Launch of Nanosatellites">
     ELaNa 35
    </a>
    (
    <a href="/wiki/Pathfinder_Technology_Demonstrator" title="Pathfinder Technology Demonstrator">
     PTD-1
    </a>
    ),
    <sup class="reference" id="cite_ref-Upcoming-ELaNa_391-2">
     <a href="#cite_note-Upcoming-ELaNa-391">
      <span class="cite-bracket">
       [
      </span>
      381
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    and multiple Kepler nanosats.
    <sup class="reference" id="cite_ref-630">
     <a href="#cite_note-630">
      <span class="cite-bracket">
       [
      </span>
      618
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-631">
     <a href="#cite_note-631">
      <span class="cite-bracket">
       [
      </span>
      619
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    D-Orbit flew their ION SCV LAURENTIUS, 10
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    satellites were placed in a polar orbit
    <sup class="reference" id="cite_ref-632">
     <a href="#cite_note-632">
      <span class="cite-bracket">
       [
      </span>
      620
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    and 2 of 15 payloads remained attached to
    <a href="/wiki/SHERPA_(space_tug)" title="SHERPA (space tug)">
     SHERPA-FX1
    </a>
    .
    <a href="/wiki/Exolaunch" title="Exolaunch">
     Exolaunch
    </a>
    deployed several small satellites and cubesats via their own deployment mechanisms. First flight of a Falcon 9 with a
    <a href="/wiki/SHERPA_(space_tug)" title="SHERPA (space tug)">
     SHERPA-FX
    </a>
    transfer stage called  SHERPA-FX1.
    <sup class="reference" id="cite_ref-633">
     <a href="#cite_note-633">
      <span class="cite-bracket">
       [
      </span>
      621
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-634">
     <a href="#cite_note-634">
      <span class="cite-bracket">
       [
      </span>
      622
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    107
   </th>
   <td>
    4 February 2021
    <br/>
    06:19
    <sup class="reference" id="cite_ref-635">
     <a href="#cite_note-635">
      <span class="cite-bracket">
       [
      </span>
      623
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5 ♺
    </a>
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1060" title="List of Falcon 9 first-stage boosters">
     B1060.5
    </a>
    <sup class="reference" id="cite_ref-636">
     <a href="#cite_note-636">
      <span class="cite-bracket">
       [
      </span>
      624
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    18 v1.0 (60 satellites)
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    This marked the fastest turnaround to date, at 27 days, and the first time a Falcon 9 flies twice within a month.
    <sup class="reference" id="cite_ref-637">
     <a href="#cite_note-637">
      <span class="cite-bracket">
       [
      </span>
      625
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    108
   </th>
   <td>
    16 February 2021
    <br/>
    03:59:37
    <sup class="reference" id="cite_ref-638">
     <a href="#cite_note-638">
      <span class="cite-bracket">
       [
      </span>
      626
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5 ♺
    </a>
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1059" title="List of Falcon 9 first-stage boosters">
     B1059.6
    </a>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    19 v1.0 (60 satellites)
    <sup class="reference" id="cite_ref-nextSL19_639-0">
     <a href="#cite_note-nextSL19-639">
      <span class="cite-bracket">
       [
      </span>
      627
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-failure" style="background: #FFC7C7; color:black; vertical-align: middle; text-align: center;">
    Failure
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    A hole in a heat-shielding engine cover, which likely developed through fatigue, allowed recirculating hot exhaust gases to damage one of the
    <a href="/wiki/SpaceX_Merlin" title="SpaceX Merlin">
     Merlin 1D
    </a>
    first-stage engines, causing it to shut down early during ascent.
    <a href="/wiki/Fail-safe" title="Fail-safe">
     Engine-out capability
    </a>
    of the Falcon 9 allowed the mission to continue and successfully deploy the 60 Starlink satellites to orbit.
    <sup class="reference" id="cite_ref-640">
     <a href="#cite_note-640">
      <span class="cite-bracket">
       [
      </span>
      628
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    The issue caused the booster to fail its landing attempt and miss the droneship
    <i>
     <a href="/wiki/Autonomous_spaceport_drone_ship#Of_Course_I_Still_Love_You" title="Autonomous spaceport drone ship">
      Of Course I Still Love You
     </a>
    </i>
    (OCISLY) after its entry burn, breaking the longest streak of 24 landing successes.
    <sup class="reference" id="cite_ref-641">
     <a href="#cite_note-641">
      <span class="cite-bracket">
       [
      </span>
      629
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    During this mission,
    <i>
     <a class="mw-redirect" href="/wiki/Ms._Tree_(ship)" title="Ms. Tree (ship)">
      GO Ms. Tree
     </a>
    </i>
    and
    <i>
     <a class="mw-redirect" href="/wiki/Ms._Chief" title="Ms. Chief">
      GO Ms. Chief
     </a>
    </i>
    were used for the last time to recover the fairings.
    <sup class="reference" id="cite_ref-642">
     <a href="#cite_note-642">
      <span class="cite-bracket">
       [
      </span>
      630
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-643">
     <a href="#cite_note-643">
      <span class="cite-bracket">
       [
      </span>
      631
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    After this mission, both ships were retired because SpaceX no longer plans to catch the fairings.
    <sup class="reference" id="cite_ref-644">
     <a href="#cite_note-644">
      <span class="cite-bracket">
       [
      </span>
      632
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    109
   </th>
   <td>
    4 March 2021
    <br/>
    08:24
    <sup class="reference" id="cite_ref-645">
     <a href="#cite_note-645">
      <span class="cite-bracket">
       [
      </span>
      633
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5 ♺
    </a>
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1049" title="List of Falcon 9 first-stage boosters">
     B1049.8
    </a>
    <sup class="reference" id="cite_ref-nextSL17_646-0">
     <a href="#cite_note-nextSL17-646">
      <span class="cite-bracket">
       [
      </span>
      634
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Kennedy_Space_Center" title="Kennedy Space Center">
     KSC
    </a>
    ,
    <br/>
    <a href="/wiki/Kennedy_Space_Center_Launch_Complex_39A" title="Kennedy Space Center Launch Complex 39A">
     LC-39A
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    17 v1.0 (60 satellites)
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Launch had previously been postponed multiple times, causing the payload Starlink L17 to launch after the L18 and L19 missions. Featured for the first time, a fairing which was flying on its fourth flight.
    <sup class="reference" id="cite_ref-647">
     <a href="#cite_note-647">
      <span class="cite-bracket">
       [
      </span>
      635
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    The second-stage deorbit burn failed, causing an uncontrolled reentry on 26 March 2021 over the west coast of the United States.
    <sup class="reference" id="cite_ref-648">
     <a href="#cite_note-648">
      <span class="cite-bracket">
       [
      </span>
      636
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    110
   </th>
   <td>
    11 March 2021
    <br/>
    08:13:29
    <sup class="reference" id="cite_ref-649">
     <a href="#cite_note-649">
      <span class="cite-bracket">
       [
      </span>
      637
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5 ♺
    </a>
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1058" title="List of Falcon 9 first-stage boosters">
     B1058.6
    </a>
    <sup class="reference" id="cite_ref-NextL20_650-0">
     <a href="#cite_note-NextL20-650">
      <span class="cite-bracket">
       [
      </span>
      638
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    20 v1.0 (60 satellites)
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Twentieth operational launch of Starlink satellites, bringing the total to 1,265 (including prototypes) launched Starlink satellites.
    <sup class="reference" id="cite_ref-651">
     <a href="#cite_note-651">
      <span class="cite-bracket">
       [
      </span>
      639
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    111
   </th>
   <td>
    14 March 2021
    <br/>
    10:01
    <sup class="reference" id="cite_ref-652">
     <a href="#cite_note-652">
      <span class="cite-bracket">
       [
      </span>
      640
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5 ♺
    </a>
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1051" title="List of Falcon 9 first-stage boosters">
     B1051.9
    </a>
   </td>
   <td>
    <a href="/wiki/Kennedy_Space_Center" title="Kennedy Space Center">
     KSC
    </a>
    ,
    <br/>
    <a href="/wiki/Kennedy_Space_Center_Launch_Complex_39A" title="Kennedy Space Center Launch Complex 39A">
     LC-39A
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    21 v1.0 (60 satellites)
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    First time a first-stage booster flew and landed for the ninth time. This flight also marked the fastest turnaround time for a fairing half, at 49 days. Both fairing halves previously flew on the Transporter-1 mission.
    <sup class="reference" id="cite_ref-653">
     <a href="#cite_note-653">
      <span class="cite-bracket">
       [
      </span>
      641
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    112
   </th>
   <td>
    24 March 2021
    <br/>
    08:28
    <sup class="reference" id="cite_ref-654">
     <a href="#cite_note-654">
      <span class="cite-bracket">
       [
      </span>
      642
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     B1060.6
    </a>
    <sup class="reference" id="cite_ref-655">
     <a href="#cite_note-655">
      <span class="cite-bracket">
       [
      </span>
      643
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    22 v1.0 (60 satellites)
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Fairing "wet recovery" achieved by contracted recovery vessel
    <i>
     Shelia Bordelon
    </i>
    for the first time. Both fairing halves were retrieved from the water.
    <sup class="reference" id="cite_ref-656">
     <a href="#cite_note-656">
      <span class="cite-bracket">
       [
      </span>
      644
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    113
   </th>
   <td>
    7 April 2021
    <br/>
    16:34
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5 ♺
    </a>
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1058" title="List of Falcon 9 first-stage boosters">
     B1058.7
    </a>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    23 v1.0 (60 satellites)
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    23rd operational launch of Starlink satellites, bringing the total to 1,385 launched
    <a href="/wiki/Starlink" title="Starlink">
     Starlink satellites
    </a>
    (including prototype). This launch featured the second fastest booster turnaround time at 27 days and 8 hours (after Starlink 18 with B1060.5, which was 4 hours faster).
    <sup class="reference" id="cite_ref-657">
     <a href="#cite_note-657">
      <span class="cite-bracket">
       [
      </span>
      645
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    114
   </th>
   <td>
    23 April 2021
    <br/>
    9:49
    <sup class="reference" id="cite_ref-658">
     <a href="#cite_note-658">
      <span class="cite-bracket">
       [
      </span>
      646
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1061" title="List of Falcon 9 first-stage boosters">
     B1061.2
    </a>
    <sup class="reference" id="cite_ref-:6_659-0">
     <a href="#cite_note-:6-659">
      <span class="cite-bracket">
       [
      </span>
      647
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Kennedy_Space_Center" title="Kennedy Space Center">
     KSC
    </a>
    ,
    <br/>
    <a href="/wiki/Kennedy_Space_Center_Launch_Complex_39A" title="Kennedy Space Center Launch Complex 39A">
     LC-39A
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX_Crew-2" title="SpaceX Crew-2">
     Crew-2
    </a>
    <br/>
    (
    <a class="mw-redirect" href="/wiki/SpaceX_Crew_Dragon_Endeavour" title="SpaceX Crew Dragon Endeavour">
     Crew Dragon C206.2
     <i>
      Endeavour
     </i>
    </a>
    ♺)
   </td>
   <td>
    ~13,000 kg (29,000 lb)
    <sup class="reference" id="cite_ref-660">
     <a href="#cite_note-660">
      <span class="cite-bracket">
       [
      </span>
      648
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a class="mw-redirect" href="/wiki/Low_Earth_Orbit" title="Low Earth Orbit">
     LEO
    </a>
    (
    <a class="mw-redirect" href="/wiki/ISS" title="ISS">
     ISS
    </a>
    )
   </td>
   <td>
    <a href="/wiki/NASA" title="NASA">
     NASA
    </a>
    (
    <a class="mw-redirect" href="/wiki/ISS_Crew_Transportation_Services" title="ISS Crew Transportation Services">
     CTS
    </a>
    )
    <sup class="reference" id="cite_ref-CCD6_509-2">
     <a href="#cite_note-CCD6-509">
      <span class="cite-bracket">
       [
      </span>
      497
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Second operational flight of Crew Dragon for Commercial Crew Program. Transported NASA astronauts
    <a href="/wiki/Shane_Kimbrough" title="Shane Kimbrough">
     Shane Kimbrough
    </a>
    and
    <a href="/wiki/K._Megan_McArthur" title="K. Megan McArthur">
     Megan McArthur
    </a>
    ,
    <a href="/wiki/JAXA" title="JAXA">
     JAXA
    </a>
    Astronaut
    <a href="/wiki/Akihiko_Hoshide" title="Akihiko Hoshide">
     Akihiko Hoshide
    </a>
    and
    <a href="/wiki/European_Space_Agency" title="European Space Agency">
     ESA
    </a>
    astronaut
    <a href="/wiki/Thomas_Pesquet" title="Thomas Pesquet">
     Thomas Pesquet
    </a>
    to the ISS.
    <sup class="reference" id="cite_ref-661">
     <a href="#cite_note-661">
      <span class="cite-bracket">
       [
      </span>
      649
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    The four astronauts will spend 6 months aboard the ISS. Beginning with the Crew-2 mission, NASA has modified the contract to allow NASA astronauts to use flight-proven Dragon capsules and booster.
    <sup class="reference" id="cite_ref-662">
     <a href="#cite_note-662">
      <span class="cite-bracket">
       [
      </span>
      650
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    Thus SpaceX reflew the Dragon used on Demo-2 and used Booster B1061-2 which had been used to launch
    <a href="/wiki/SpaceX_Crew-1" title="SpaceX Crew-1">
     Crew-1
    </a>
    in November 2020.
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    115
   </th>
   <td>
    29 April 2021
    <br/>
    03:44
    <sup class="reference" id="cite_ref-663">
     <a href="#cite_note-663">
      <span class="cite-bracket">
       [
      </span>
      651
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1060" title="List of Falcon 9 first-stage boosters">
     B1060.7
    </a>
    <sup class="reference" id="cite_ref-664">
     <a href="#cite_note-664">
      <span class="cite-bracket">
       [
      </span>
      652
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    24 v1.0 (60 satellites)
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    24th operational launch of Starlink satellites, bringing the total to 1,434 Starlink satellites still in orbit. This launch also paid tribute to
    <a href="/wiki/Apollo_11" title="Apollo 11">
     Apollo 11
    </a>
    crew
    <a href="/wiki/Michael_Collins_(astronaut)" title="Michael Collins (astronaut)">
     Michael Collins
    </a>
    , who passed away hours before the launch.
    <sup class="reference" id="cite_ref-665">
     <a href="#cite_note-665">
      <span class="cite-bracket">
       [
      </span>
      653
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    116
   </th>
   <td>
    4 May 2021
    <br/>
    19:01
    <sup class="reference" id="cite_ref-666">
     <a href="#cite_note-666">
      <span class="cite-bracket">
       [
      </span>
      654
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1049" title="List of Falcon 9 first-stage boosters">
     B1049.9
    </a>
    <sup class="reference" id="cite_ref-667">
     <a href="#cite_note-667">
      <span class="cite-bracket">
       [
      </span>
      655
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Kennedy_Space_Center" title="Kennedy Space Center">
     KSC
    </a>
    ,
    <br/>
    <a href="/wiki/Kennedy_Space_Center_Launch_Complex_39A" title="Kennedy Space Center Launch Complex 39A">
     LC-39A
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    25 v1.0 (60 satellites)
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    25th operational launch of Starlink satellites, bringing the total to 1,494 Starlink satellites still in orbit, second time a booster flew for the ninth time.
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    117
   </th>
   <td>
    9 May 2021
    <br/>
    06:42
    <sup class="reference" id="cite_ref-668">
     <a href="#cite_note-668">
      <span class="cite-bracket">
       [
      </span>
      656
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1051" title="List of Falcon 9 first-stage boosters">
     B1051.10
    </a>
    <sup class="reference" id="cite_ref-669">
     <a href="#cite_note-669">
      <span class="cite-bracket">
       [
      </span>
      657
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    27 v1.0 (60 satellites)
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Booster flew and landed for a record 10th time, with reused fairings, bringing the total number of operational Starlink satellites in the first shell to approximately 1516 out of a planned 1584.
    <sup class="reference" id="cite_ref-NSF8521_670-0">
     <a href="#cite_note-NSF8521-670">
      <span class="cite-bracket">
       [
      </span>
      658
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    118
   </th>
   <td>
    15 May 2021
    <br/>
    22:56
    <sup class="reference" id="cite_ref-671">
     <a href="#cite_note-671">
      <span class="cite-bracket">
       [
      </span>
      659
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1058" title="List of Falcon 9 first-stage boosters">
     B1058.8
    </a>
    <sup class="reference" id="cite_ref-672">
     <a href="#cite_note-672">
      <span class="cite-bracket">
       [
      </span>
      660
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Kennedy_Space_Center" title="Kennedy Space Center">
     KSC
    </a>
    ,
    <br/>
    <a href="/wiki/Kennedy_Space_Center_Launch_Complex_39A" title="Kennedy Space Center Launch Complex 39A">
     LC-39A
    </a>
    <sup class="reference" id="cite_ref-673">
     <a href="#cite_note-673">
      <span class="cite-bracket">
       [
      </span>
      661
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    26 v1.0 (52 Satellites)
    <br/>
    <a href="/wiki/Capella_Space" title="Capella Space">
     Capella
    </a>
    -6 &amp;
    <a href="/wiki/Tyvak" title="Tyvak">
     Tyvak
    </a>
    -0130
    <sup class="reference" id="cite_ref-674">
     <a href="#cite_note-674">
      <span class="cite-bracket">
       [
      </span>
      662
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    ~14,000 kg (31,000 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
    <br/>
    <a href="/wiki/Capella_Space" title="Capella Space">
     Capella Space
    </a>
    and
    <a href="/wiki/Tyvak" title="Tyvak">
     Tyvak
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Rideshare launch with a targeted orbit at 569x582, significantly higher than typical Starlink launches, to allow for needs of the rideshare payloads.
    <sup class="reference" id="cite_ref-SL28_675-0">
     <a href="#cite_note-SL28-675">
      <span class="cite-bracket">
       [
      </span>
      663
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    119
   </th>
   <td>
    26 May 2021
    <br/>
    18:59
    <sup class="reference" id="cite_ref-676">
     <a href="#cite_note-676">
      <span class="cite-bracket">
       [
      </span>
      664
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1063" title="List of Falcon 9 first-stage boosters">
     B1063.2
    </a>
    <sup class="reference" id="cite_ref-NSFSl28_677-0">
     <a href="#cite_note-NSFSl28-677">
      <span class="cite-bracket">
       [
      </span>
      665
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
    <sup class="reference" id="cite_ref-NSFSl28_677-1">
     <a href="#cite_note-NSFSl28-677">
      <span class="cite-bracket">
       [
      </span>
      665
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Starlink" title="Starlink">
     Starlink
    </a>
    28 v1.0 (60 Satellites)
    <sup class="reference" id="cite_ref-NSFSl28_677-2">
     <a href="#cite_note-NSFSl28-677">
      <span class="cite-bracket">
       [
      </span>
      665
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    15,600 kg (34,400 lb)
   </td>
   <td>
    <a href="/wiki/Low_Earth_orbit" title="Low Earth orbit">
     LEO
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX" title="SpaceX">
     SpaceX
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Will likely complete the first shell of the Starlink network located at 550 km altitude and containing 1584 satellites.
    <sup class="reference" id="cite_ref-SL28_675-1">
     <a href="#cite_note-SL28-675">
      <span class="cite-bracket">
       [
      </span>
      663
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    It was 40th launch a fairing was reused, with one half being used for the 5th time (first fairing to do so) and the other for a 3rd time.
    <sup class="reference" id="cite_ref-sn20210526_678-0">
     <a href="#cite_note-sn20210526-678">
      <span class="cite-bracket">
       [
      </span>
      666
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    This launch marks SpaceX's 100th successful launch in a row without in-flight failure since December 2015.
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    120
   </th>
   <td>
    3 June 2021
    <br/>
    17:29
    <sup class="reference" id="cite_ref-679">
     <a href="#cite_note-679">
      <span class="cite-bracket">
       [
      </span>
      667
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1067" title="List of Falcon 9 first-stage boosters">
     B1067.1
    </a>
    <sup class="reference" id="cite_ref-680">
     <a href="#cite_note-680">
      <span class="cite-bracket">
       [
      </span>
      668
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Kennedy_Space_Center" title="Kennedy Space Center">
     KSC
    </a>
    ,
    <br/>
    <a href="/wiki/Kennedy_Space_Center_Launch_Complex_39A" title="Kennedy Space Center Launch Complex 39A">
     LC-39A
    </a>
   </td>
   <td>
    <a href="/wiki/SpaceX_CRS-22" title="SpaceX CRS-22">
     SpaceX CRS-22
    </a>
    <br/>
    (
    <a href="/wiki/Cargo_Dragon_C209" title="Cargo Dragon C209">
     Dragon C209.1
    </a>
    )
   </td>
   <td>
    3,328 kg (7,337 lb)
   </td>
   <td>
    <a class="mw-redirect" href="/wiki/Low_Earth_Orbit" title="Low Earth Orbit">
     LEO
    </a>
    (
    <a class="mw-redirect" href="/wiki/ISS" title="ISS">
     ISS
    </a>
    )
   </td>
   <td>
    <a href="/wiki/NASA" title="NASA">
     NASA
    </a>
    (
    <a href="/wiki/Commercial_Resupply_Services" title="Commercial Resupply Services">
     CRS
    </a>
    )
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    Second of a minimum of six new cargo missions under the
    <a href="/wiki/Commercial_Resupply_Services" title="Commercial Resupply Services">
     CRS-2 contract
    </a>
    , which NASA awarded SpaceX in 2015. Mission was flown with an uncrewed Dragon 2 capsule,
    <sup class="reference" id="cite_ref-nasa-20160114_681-0">
     <a href="#cite_note-nasa-20160114-681">
      <span class="cite-bracket">
       [
      </span>
      669
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    which carried solar panels, catalytic reactor for the station’s life support system, an emergency air supply system, Kurs remote control unit, and a Potable Water Dispense (PWD) filter. Also carried were the RamSat cubesat as payload for
    <a class="mw-redirect" href="/wiki/ELaNa" title="ELaNa">
     ELaNa 36
    </a>
    ,
    <sup class="reference" id="cite_ref-682">
     <a href="#cite_note-682">
      <span class="cite-bracket">
       [
      </span>
      670
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    the SOAR cubesat for the
    <a href="/wiki/University_of_Manchester" title="University of Manchester">
     University of Manchester
    </a>
    <sup class="reference" id="cite_ref-683">
     <a href="#cite_note-683">
      <span class="cite-bracket">
       [
      </span>
      671
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    and the first Mauritian satellite MIR-SAT1
    <sup class="reference" id="cite_ref-684">
     <a href="#cite_note-684">
      <span class="cite-bracket">
       [
      </span>
      672
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    to be launched from the station later.
   </td>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    121
   </th>
   <td>
    6 June 2021
    <br/>
    04:26
    <sup class="reference" id="cite_ref-685">
     <a href="#cite_note-685">
      <span class="cite-bracket">
       [
      </span>
      673
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    <a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">
     F9 B5
    </a>
    ♺
    <br/>
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1061" title="List of Falcon 9 first-stage boosters">
     B1061.3
    </a>
   </td>
   <td>
    <a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">
     CCSFS
    </a>
    ,
    <br/>
    <a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">
     SLC-40
    </a>
   </td>
   <td>
    <a href="/wiki/Sirius_XM#Satellites" title="Sirius XM">
     SXM-8
    </a>
    <sup class="reference" id="cite_ref-spacex_manifest_686-0">
     <a href="#cite_note-spacex_manifest-686">
      <span class="cite-bracket">
       [
      </span>
      674
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
   <td>
    7,000 kg (15,000 lb)
   </td>
   <td>
    <a href="/wiki/Geostationary_transfer_orbit" title="Geostationary transfer orbit">
     GTO
    </a>
   </td>
   <td>
    <a href="/wiki/Sirius_XM" title="Sirius XM">
     Sirius XM
    </a>
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
   </td>
   <td class="table-success" style="background: #9EFF9E; color:black; vertical-align: middle; text-align: center;">
    Success
    <br/>
    <small>
     <span class="nowrap">
      (drone ship)
     </span>
    </small>
   </td>
  </tr>
  <tr>
   <td colspan="9">
    A large, high-power broadcasting satellite for SiriusXM's digital audio radio service (DARS) contracted together with SXM-7 to replace the aging XM-4 satellite and allow broadcast to radios without the need for large dish-type antennas on the ground.
    <sup class="reference" id="cite_ref-auto3_612-1">
     <a href="#cite_note-auto3-612">
      <span class="cite-bracket">
       [
      </span>
      600
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
    <sup class="reference" id="cite_ref-687">
     <a href="#cite_note-687">
      <span class="cite-bracket">
       [
      </span>
      675
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </td>
  </tr>
 </tbody>
</table>


In [ ]:
# Here we create an empty dictionary to store the extracted data
## The keys are the extracted column names above
launch_dict= dict.fromkeys(column_names)

# Remove irrelevant column
del launch_dict['Date and time ( )']

# Initialize the launch_dict
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Add some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

# Herein we fill the launch_dict with data from the HTML tables
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading corresponds to a launch number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)
            print(f"\n\nFlight number: ",flight_number)
            datatimelist = date_time(row[0])
            
            # Date value
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            print("Date: ",date)
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'].append(date)
            print("Time: ",time)
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            print("Version Booster",bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            print("Launch site",launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            print("Payload: ",payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            print("Payload mass: ",payload_mass)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            print("Orbit: ",orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else row[6].string
            launch_dict['Customer'].append(customer)
            print("Customer: ",customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            print("Launch outcome: ",launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            print("Booster landing: ",booster_landing)


# We now use launch_dict to create a dataframe
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
df.head(50)



Flight number:  1
Date:  4 June 2010
Time:  18:45
Version Booster F9 v1.07B0003.18
Launch site CCAFS
Payload:  Dragon Spacecraft Qualification Unit
Payload mass:  0
Orbit:  LEO
Customer:  SpaceX
Launch outcome:  Success

Booster landing:  Failure


Flight number:  2
Date:  8 December 2010
Time:  15:43
Version Booster F9 v1.07B0004.18
Launch site CCAFS
Payload:  Dragon
Payload mass:  0
Orbit:  LEO
Customer:  NASA
Launch outcome:  Success
Booster landing:  Failure


Flight number:  3
Date:  22 May 2012
Time:  07:44
Version Booster F9 v1.07B0005.18
Launch site CCAFS
Payload:  Dragon
Payload mass:  525 kg
Orbit:  LEO
Customer:  NASA
Launch outcome:  Success
Booster landing:  No attempt



Flight number:  4
Date:  8 October 2012
Time:  00:35
Version Booster F9 v1.07B0006.18
Launch site CCAFS
Payload:  SpaceX CRS-1
Payload mass:  4,700 kg
Orbit:  LEO
Customer:  NASA
Launch outcome:  Success

Booster landing:  No attempt


Flight number:  5
Date:  1 March 2013
Time:  15:10
Version Booster F

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.07B0003.18,booster_landing,4 June 2010,4 June 2010
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,booster_landing,8 December 2010,8 December 2010
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,booster_landing,22 May 2012,22 May 2012
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.07B0006.18,booster_landing,8 October 2012,8 October 2012
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.07B0007.18,booster_landing,1 March 2013,1 March 2013
5,6,VAFB,CASSIOPE,500 kg,Polar orbit,MDA,Success,F9 v1.17B10038,booster_landing,29 September 2013,29 September 2013
6,7,CCAFS,SES-8,"3,170 kg",GTO,SES,Success,F9 v1.1,booster_landing,3 December 2013,3 December 2013
7,8,CCAFS,Thaicom 6,"3,325 kg",GTO,Thaicom,Success,F9 v1.1,booster_landing,6 January 2014,6 January 2014
8,9,Cape Canaveral,SpaceX CRS-3,"2,296 kg",LEO,NASA,Success\n,F9 v1.1,booster_landing,18 April 2014,18 April 2014
9,10,Cape Canaveral,Orbcomm-OG2,"1,316 kg",LEO,Orbcomm,Success,F9 v1.1,booster_landing,14 July 2014,14 July 2014


In [20]:
import re
def clean_Version_Booster(Version):
    """ cleans the 'Version Booster' column"""
    Version = Version.apply(lambda x: re.sub(r'[^\w\s\.]', '', str(x)))
    return Version


df['Version Booster'] = clean_Version_Booster(df['Version Booster'])
#df['Version Booster'] = df['Version Booster'].str.strip()

df['Version Booster'].value_counts()

Version Booster
F9 B5                 36
F9 FT                 18
F9 v1.1               14
F9 B4                  7
F9 v1.07B0004.18       1
F9 v1.07B0003.18       1
F9 v1.17B10038         1
F9 v1.07B0007.18       1
F9 v1.07B0006.18       1
F9 v1.07B0005.18       1
F9 FTB1031.2220        1
F9 FTB1035.2227        1
F9 FTB1036.2227        1
F9 FTB1032.2245        1
F9 FTB1038.2268        1
F9 B4B1041.2268        1
F9 B4B1039.2292        1
F9 FTB1029.2195        1
F9 B5311B1046.1268     1
F9 B4B1043.2322        1
F9 B4B1040.2268        1
F9 B4B1045.2336        1
F9 B5349B1048          1
F9 B5B1046.2354        1
F9 B5B1048.2364        1
F9 B5B1047.2268        1
F9 B5B1046.3268        1
F9 B5B1049.2397        1
F9 B5B1048.3399        1
F9 B5413               1
F9 B5B1049.3434        1
F9 B5B1051.2420        1
F9 B5B1056.2465        1
F9 B5B1047.3472        1
F9 B5B1056.3482        1
F9 B5B1058.2544        1
F9 B5B1049.6544        1
F9 B5B1060.2563        1
F9 B5B1058.3565        1
F9 B5B105

In [12]:
# We also clean the \n inside Launch Outcome column

def clean_Launch_Outcome(Launch_Outcome):
    Launch_Outcome = Launch_Outcome.apply(lambda x: re.sub(r'[^\w\s\.]', '', str(x)))
    # removes occurrences of \n
    Launch_Outcome = Launch_Outcome.str.replace('\n', '')
    return Launch_Outcome


df['Launch outcome'] = clean_Launch_Outcome(df['Launch outcome'])
df['Launch outcome'].value_counts()

Launch outcome
Success    120
Failure      1
Name: count, dtype: int64

## Try non-static urls

In [2]:
import os

# move to parent folder ..
os.chdir("..")
print("Path is moved to project root")

Path is moved to project root


In [3]:
from src.utils.config_loader import load_config
from src.collect_web import extract_table_headers, fetch_html

config = load_config()
url_1=config["web_url_1"]
url_2=config["web_url_2"]


html_1 = fetch_html(url_1)
html_2 = fetch_html(url_2)

column_names_1, soup_1 = extract_table_headers(html_1, table_index=2, return_soup=True)
print(column_names_1)
column_names_2, soup_2 = extract_table_headers(html_2, table_index=2, return_soup=True)
print(column_names_2)

html_tables_1 = soup_1.find_all(name = 'table')
print(f"\nFound {len(html_tables_1)} tables on the page")

html_tables_2 = soup_2.find_all(name = 'table')
print(f"\nFound {len(html_tables_2)} tables on the page")


Fetching HTML from https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2010%E2%80%932019)
Fetching HTML from https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2020%E2%80%932022)
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'FH 4']

Found 17 tables on the page

Found 13 tables on the page


In [4]:
first_launch_table_1= html_tables_1[0]
last_launch_table_1= html_tables_1[6]
# prints output and put it inside a markdown
print(last_launch_table_1.prettify())

<table class="wikitable plainrowheaders sticky-header collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    <br/>
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     booster
    </a>
    <sup class="reference" id="cite_ref-booster_5-6">
     <a href="#cite_note-booster-5">
      <span class="cite-bracket">
       [
      </span>
      a
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch
    <br/>
    site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_43-5">
     <a href="#cite_note-Dragon-43">
      <span class="cite-bracket">
       [
      </span>
      c
      <sp

In [ ]:
first_launch_table_2= html_tables_2[0]
last_launch_table_2= html_tables_2[2]
# prints output and put it inside a markdown
print(last_launch_table_2.prettify())

In [15]:
from src.collect_web import extract_date_time, extract_booster_version, extract_landing_status, extract_mass

def parse_soup(column_names, soup):
    """
    Parse a BeautifulSoup object and extract launch data from its tables.
    """
    
    # create empty dictionary with keys from the extracted column names in the previous task
    launch_dict= dict.fromkeys(column_names)

    # Remove irrelevant column
    del launch_dict['Date and time ( )']

    # Initialize the launch_dict with each value to be an empty list
    launch_dict['Flight No.'] = []
    launch_dict['Launch site'] = []
    launch_dict['Payload'] = []
    launch_dict['Payload mass'] = []
    launch_dict['Orbit'] = []
    launch_dict['Customer'] = []
    launch_dict['Launch outcome'] = []

    # Add some new columns
    launch_dict['Version Booster']=[]
    launch_dict['Booster landing']=[]
    launch_dict['Date']=[]
    launch_dict['Time']=[]

    # NEXT: Fill up the `launch_dict` with launch records extracted from table rows
    extracted_row = 0
    print(f"Found {len(soup.find_all('table'))} tables on the page")
    #Extract each table 
    for table_number,table in enumerate(soup.find_all('table')):
        print(f"Extracting table {table_number} out of {len(soup.find_all('table'))}")
        # get table row 
        for rows in table.find_all("tr"):

            # Default state
            flag = False
            flight_number = None  # Initialize to avoid 'undefined'

            #check to see if first table heading is as number corresponding to launch a number 
            if rows.th:
                if rows.th.string:
                    flight_number=rows.th.string.strip()
                    flag=flight_number.isdigit()
                else:
                    flag=False
            else:
                flag=False
            #get table element 
            row=rows.find_all('td')
            #if it is number save cells in a dictonary 
            if flag:
                extracted_row += 1
                # Flight Number value
                launch_dict['Flight No.'].append(flight_number)
                print('Flight no.: ', flight_number)
            
                # Date value
                datatimelist = extract_date_time(row[0])
                date = datatimelist[0].strip(',')
                launch_dict['Date'].append(date)
                print(date)
            
                # Time value
                time = datatimelist[1]
                launch_dict['Time'].append(date)
                #print(time)
              
                # Booster version
                bv=extract_booster_version(row[1])
                if not(bv):
                    bv=row[1].a.string
                launch_dict['Version Booster'].append(bv)
                print(bv)
            
                # Launch Site
                launch_site = row[2].a.string
                launch_dict['Launch site'].append(launch_site)
                print(launch_site)
            
                # Payload
                payload = row[3].a.string
                launch_dict['Payload'].append(payload)
                print(payload)
            
                # Payload Mass
                payload_mass = extract_mass(row[4])
                launch_dict['Payload mass'].append(payload_mass)
                print(payload_mass)
            
                # Orbit
                orbit = row[5].a.string
                launch_dict['Orbit'].append(orbit)
                print(orbit)
            
                # Customer
                customer = row[6].a.string if row[6].a else row[6].string
                launch_dict['Customer'].append(customer)
                print(customer)
            
                # Launch outcome
                launch_outcome = list(row[7].strings)[0]
                launch_outcome = launch_outcome.replace('\n', '')
                launch_dict['Launch outcome'].append(launch_outcome)
                print(launch_outcome)
            
                # Booster landing
                booster_landing = extract_landing_status(row[8])
                booster_landing = booster_landing.replace('\n', '')
                launch_dict['Booster landing'].append(booster_landing)
                print(booster_landing)

            print(f"{extracted_row} records extracted\n") if flag else None

    return launch_dict


launch_dict_1 = parse_soup(column_names_1, soup_1)
launch_dict_2 = parse_soup(column_names_2, soup_2)


Found 17 tables on the page
Extracting table 0 out of 17
Flight no.:  1
4 June 2010
F9v1.0
Cape Canaveral
Dragon Spacecraft Qualification Unit
U
LEO
SpaceX
Success
Failure (parachute)
1 records extracted

Flight no.:  2
8 December 2010
F9v1.0
Cape Canaveral
SpaceX COTS Demo Flight 1
U
LEO
NASA
Success
Failure (parachute)
2 records extracted

Flight no.:  3
22 May 2012
F9 v1.0
Cape Canaveral
SpaceX COTS Demo Flight 2
525 kg
LEO
NASA
Success
No attempt
3 records extracted

Flight no.:  4
8 October 2012
F9 v1.0
Cape Canaveral
SpaceX CRS-1
4,700 kg
LEO
NASA
Success
No attempt
4 records extracted

Flight no.:  5
1 March 2013
F9 v1.0
Cape Canaveral
SpaceX CRS-2
4,877 kg
LEO
NASA
Success
No attempt
5 records extracted

Flight no.:  6
29 September 2013
F9 v1.1
Vandenberg
CASSIOPE
500 kg
Polar orbit
MDA
Success
Failure (ocean)
6 records extracted

Flight no.:  7
3 December 2013
F9 v1.1
Cape Canaveral
SES-8
3,170 kg
GTO
SES
Success
No attempt
7 records extracted

Extracting table 1 out of 17
Fli

In [16]:
import pandas as pd
df_1 = pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict_1.items() })
df_1.head(20)

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,Cape Canaveral,Dragon Spacecraft Qualification Unit,U,LEO,SpaceX,Success,F9v1.0,Failure (parachute),4 June 2010,4 June 2010
1,2,Cape Canaveral,SpaceX COTS Demo Flight 1,U,LEO,NASA,Success,F9v1.0,Failure (parachute),8 December 2010,8 December 2010
2,3,Cape Canaveral,SpaceX COTS Demo Flight 2,525 kg,LEO,NASA,Success,F9 v1.0,No attempt,22 May 2012,22 May 2012
3,4,Cape Canaveral,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.0,No attempt,8 October 2012,8 October 2012
4,5,Cape Canaveral,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.0,No attempt,1 March 2013,1 March 2013
5,6,Vandenberg,CASSIOPE,500 kg,Polar orbit,MDA,Success,F9 v1.1,Failure (ocean),29 September 2013,29 September 2013
6,7,Cape Canaveral,SES-8,"3,170 kg",GTO,SES,Success,F9 v1.1,No attempt,3 December 2013,3 December 2013
7,8,Cape Canaveral,Thaicom 6,"3,325 kg",GTO,Thaicom,Success,F9 v1.1,No attempt,6 January 2014,6 January 2014
8,9,Cape Canaveral,SpaceX CRS-3,"2,296 kg",LEO,NASA,Success,F9 v1.1,Controlled (ocean),18 April 2014,18 April 2014
9,10,Cape Canaveral,Orbcomm-OG2,"1,316 kg",LEO,Orbcomm,Success,F9 v1.1,Controlled (ocean),14 July 2014,14 July 2014


In [17]:
df_2 = pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict_2.items() })
df_2.tail(20)


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,FH 4,Version Booster,Booster landing,Date,Time
97,175,Kennedy,Starlink,"~11,938 kg",LEO,SpaceX,Success,NaN,F9 B5,Success (,11 September 2022,11 September 2022
98,176,Cape Canaveral,Starlink,"~16,700 kg",LEO,SpaceX,Success,NaN,F9 B5[,Success (,19 September 2022,19 September 2022
99,177,Cape Canaveral,Starlink,"~16,100 kg",LEO,SpaceX,Success,NaN,F9 B5[,Success (,24 September 2022,24 September 2022
100,178,Kennedy,Crew-5,"~13,000 kg",LEO,NASA,Success,NaN,F9 B5,Success (,5 October 2022,5 October 2022
101,179,Vandenberg,Starlink,"~16,100 kg",LEO,SpaceX,Success,NaN,F9 B5,Success (,5 October 2022,5 October 2022
102,180,Cape Canaveral,Galaxy 33 & 34,"7,350 kg",GTO,Intelsat,Success,NaN,F9 B5,Success (,8 October 2022,8 October 2022
103,181,Cape Canaveral,Hotbird 13F,"~4,500 kg",GTO,Eutelsat,Success,NaN,F9 B5,Success (,15 October 2022,15 October 2022
104,182,Cape Canaveral,Starlink,"~16,700 kg",LEO,SpaceX,Success,NaN,F9 B5[,Success (,20 October 2022,20 October 2022
105,183,Vandenberg,Starlink,"~16,400 kg",LEO,SpaceX,Success,NaN,F9 B5,Success (,28 October 2022,28 October 2022
106,184,Cape Canaveral,Hotbird 13G,"~4,500 kg",GTO,Eutelsat,Success,NaN,F9 B5,Success (,3 November 2022,3 November 2022


In [13]:
# Combine dataframe on Flight No.
df_combined = pd.concat([df_1, df_2], ignore_index=True)
df_combined.head(100)

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time,FH 4
0,1,Cape Canaveral,Dragon Spacecraft Qualification Unit,U,LEO,SpaceX,Success,F9v1.0,Failure (parachute)\n,4 June 2010,4 June 2010,NaN
1,2,Cape Canaveral,SpaceX COTS Demo Flight 1,U,LEO,NASA,Success,F9v1.0,Failure (parachute)\n,8 December 2010,8 December 2010,NaN
2,3,Cape Canaveral,SpaceX COTS Demo Flight 2,525 kg,LEO,NASA,Success,F9 v1.0,No attempt\n,22 May 2012,22 May 2012,NaN
3,4,Cape Canaveral,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.0,No attempt,8 October 2012,8 October 2012,NaN
4,5,Cape Canaveral,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.0,No attempt\n,1 March 2013,1 March 2013,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Cape Canaveral,Starlink,"15,600 kg",LEO,SpaceX,Success,F9 B5,Success (,24 October 2020,24 October 2020,NaN
96,97,Cape Canaveral,GPS III,"4,311 kg",MEO,USSF,Success,F9 B5,Success (,5 November 2020,5 November 2020,NaN
97,98,Kennedy,Crew-1,"~12,500 kg",LEO,NASA,Success,F9 B5B1061.195,Success (,16 November 2020,16 November 2020,NaN
98,99,Vandenberg,Sentinel-6 Michael Freilich (Jason-CS A),"1,192 kg",LEO,NASA,Success,F9 B5,Success (,21 November 2020,21 November 2020,NaN


In [22]:
API_LIMIT_DATE = '2020-12-01'
print(type(API_LIMIT_DATE.split('-')))
print(API_LIMIT_DATE.split('-'))

year, month, day = API_LIMIT_DATE.split('-')
print(int(year), int(month), int(day))

<class 'list'>
['2020', '12', '01']
2020 12 1
